In [1]:
## import all necessary packages
import time
import json
import re

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict


In [2]:
from happyfuntokenizing import Tokenizer

** load data **

In [3]:
with open('./data/tweets_1M.json','r') as f:
    tweets = json.load(f)

In [4]:
#subset_tweets[:5]
tweet_df = DataFrame(tweets)
tweet_df[30:40]

,id,lat,lng,text,timeStamp,user_id
30,377653831934705664,34.125987,-117.990605,😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth,Wed Sep 11 04:44:24 +0000 2013,108769611
31,377653836804288512,36.920730,-121.744991,@hello_emmelie you know girl. Emmelie let me s...,Wed Sep 11 04:44:25 +0000 2013,1049702996
32,377653839312465920,37.723981,-122.484290,In the 90's actresses didn't wear bras... #cha...,Wed Sep 11 04:44:26 +0000 2013,38926916
33,377653845650051072,37.783569,-122.212397,""" you deserve your name on a crown , on a thro...",Wed Sep 11 04:44:27 +0000 2013,230608737
34,377653846467952640,37.717741,-122.153800,Thank goodness for CPT days 👍,Wed Sep 11 04:44:28 +0000 2013,876663008
35,377653847717871616,34.185616,-116.399403,Yay my cat is cuddling with me tonight❤,Wed Sep 11 04:44:28 +0000 2013,526041822
36,377653854109978624,37.348083,-120.569948,Fake people suck...😖,Wed Sep 11 04:44:29 +0000 2013,28348907
37,377653857146634240,37.620391,-122.090301,Lol now that's it's winter almost... hahaa...,Wed Sep 11 04:44:30 +0000 2013,814226478
38,377653857394098176,37.577402,-122.323459,I tried to change the subject so I said some c...,Wed Sep 11 04:44:30 +0000 2013,1329849811
39,377653857364758530,37.758516,-121.980800,@teamroshe just made my day better thanks guys...,Wed Sep 11 04:44:30 +0000 2013,232122031


** subset to just column with the actual tweet **

In [5]:
text_only = tweet_df.text
text_only[:10]

0    @Tanner_Cortez hey checkout the website: http:...
1                                          @Lennayy7 😪
2                         i laugh a lot with that line
3                 sons of anarchy is back on woop woop
4    Drinking a Fresh Squeezed IPA by @deschutesbee...
5    I have 8 am classes this quarter ... I need to...
6               Why is Milgrim's eyes always red af..?
7    #FightOn! :-P MT @PoliticsofJon: Eh, you're do...
8    Asked him what he did today and got a response...
9                       @Dlop24 my moms just nosey lol
Name: text, dtype: object

In [6]:
tweet_df.text[:10][1]

'@Lennayy7 😪'

** find all tweets with an emoji... used hadrien's link to emoji engn part 2 but only used part of the
snippet... **

In [61]:
try:
    # Wide UCS-4 build
    highpoints = re.compile(u'['
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\u2600-\u26FF\u2700-\u27BF]+', 
        re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    highpoints = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)

In [62]:
emoji_list = []
for index, value in enumerate(text_only):
    if highpoints.search(value):
        emoji_list.append((index, value))

In [63]:
text_only[1]

'@Lennayy7 😪'

In [64]:
(emoji_list)

[(1, '@Lennayy7 😪'),
 (11, '@Serina_Zarro @BigTimeFlow TWINS🙈'),
 (30, '😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth'),
 (33, '" you deserve your name on a crown , on a throne" 👑❤'),
 (34, 'Thank goodness for CPT days 👍'),
 (35, 'Yay my cat is cuddling with me tonight❤'),
 (36, 'Fake people suck...😖'),
 (42, 'So stop✋'),
 (44,
  "@chunkie_tiffie IK I watched your guys' first set. You guys are so raw 👏🙌"),
 (48, "Alright I think I'm good... 😂"),
 (49, "iOS 7 doesn't come out for another week. Liars. 😒"),
 (52, '@Tonyy_Spitta thank you! 😊'),
 (54, 'Will workout for pleasure 😌'),
 (55, "You know it's real😂😉😏 @sukiibasma7 http://t.co/POjaiZ8Th3"),
 (59, '@DianaMichelleee 😂😂😂😂😂'),
 (65, 'Who would of thought a girl like be would double as superstar! 🎧🎤'),
 (66, 'It is almost 9/11 😞'),
 (67,
  'Had the most amazing night at Dave with the most amazing guy. ❤ shnozzykev @ Shoreline Amphitheatre http://t.co/JENHyoNhr4'),
 (78, 'Lmaoooo. 😂😂😂😂'),
 (79, '“@IDAWBA: @Elazay_xo zaaaauuuuuummm” whatch

In [65]:
(emoji_list)[:10]

[(1, '@Lennayy7 😪'),
 (11, '@Serina_Zarro @BigTimeFlow TWINS🙈'),
 (30, '😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth'),
 (33, '" you deserve your name on a crown , on a throne" 👑❤'),
 (34, 'Thank goodness for CPT days 👍'),
 (35, 'Yay my cat is cuddling with me tonight❤'),
 (36, 'Fake people suck...😖'),
 (42, 'So stop✋'),
 (44,
  "@chunkie_tiffie IK I watched your guys' first set. You guys are so raw 👏🙌"),
 (48, "Alright I think I'm good... 😂")]

** subset dataframe to rows with emojis **

In [66]:
emoji_index = [x[0] for x in emoji_list]

In [67]:
emoji_df = tweet_df.ix[emoji_index]

In [68]:
emoji_df

,id,lat,lng,text,timeStamp,user_id
1,377652255346159616,34.087406,-117.462604,@Lennayy7 😪,Wed Sep 11 04:38:08 +0000 2013,312179473
11,377652290297278464,38.719410,-121.354616,@Serina_Zarro @BigTimeFlow TWINS🙈,Wed Sep 11 04:38:17 +0000 2013,363417284
30,377653831934705664,34.125987,-117.990605,😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth,Wed Sep 11 04:44:24 +0000 2013,108769611
33,377653845650051072,37.783569,-122.212397,""" you deserve your name on a crown , on a thro...",Wed Sep 11 04:44:27 +0000 2013,230608737
34,377653846467952640,37.717741,-122.153800,Thank goodness for CPT days 👍,Wed Sep 11 04:44:28 +0000 2013,876663008
35,377653847717871616,34.185616,-116.399403,Yay my cat is cuddling with me tonight❤,Wed Sep 11 04:44:28 +0000 2013,526041822
36,377653854109978624,37.348083,-120.569948,Fake people suck...😖,Wed Sep 11 04:44:29 +0000 2013,28348907
42,377653861907185664,34.039974,-118.077466,So stop✋,Wed Sep 11 04:44:31 +0000 2013,1132521368
44,377653883793076224,37.653524,-122.443788,@chunkie_tiffie IK I watched your guys' first ...,Wed Sep 11 04:44:37 +0000 2013,572272257
48,377653893515464704,39.275773,-121.666600,Alright I think I'm good... 😂,Wed Sep 11 04:44:39 +0000 2013,387589131


There's a user id so technically we could identify sets of tweets by users. 

** first stab at counting frequency of different emojis **

In [69]:
from nltk.tokenize import word_tokenize
# tknzr = TweetTokenizer()
s0 = u"@Serina_Zarro @BigTimeFlow TWINS\U0001f648"
word_tokenize(s0)

['@', 'Serina_Zarro', '@', 'BigTimeFlow', 'TWINS🙈']

In [70]:
emoji_df.shape

(174231, 6)

### Counting number of emojis per text

In [72]:
def count_emojis(text):
    return len(highpoints.findall(text))
    

In [73]:
emoji_df["Count emojis"] = emoji_df["text"].apply(count_emojis)
emoji_df["text length"] = emoji_df["text"].apply(lambda x: len(x))
emoji_df.describe()

,id,lat,lng,user_id,Count emojis,text length
count,1.742310e+05,174231.000000,174231.000000,1.742310e+05,174231.000000,174231.000000
mean,3.822490e+17,37.120009,-121.321720,5.387007e+08,1.128835,57.858682
std,2.379545e+15,1.490273,1.622052,4.569804e+08,0.639105,34.683764
min,3.776523e+17,32.719729,-124.315800,7.180000e+02,1.000000,1.000000
25%,3.801770e+17,37.251564,-122.221305,1.765883e+08,1.000000,30.000000
50%,3.823081e+17,37.591110,-121.926706,4.042137e+08,1.000000,50.000000
75%,3.843769e+17,37.790582,-121.628003,7.818799e+08,1.000000,80.000000
max,3.863382e+17,41.985420,-114.839596,1.934553e+09,70.000000,221.000000


Most people who use emoji use 1 or 2 with median 52 characters. 

In [74]:
emoji_df[emoji_df["Count emojis"] == 140]

,id,lat,lng,text,timeStamp,user_id,Count emojis,text length


## DISTRIBUTION OF ALL EMOJIS

In [75]:
def just_emojis(text):
    return (highpoints.findall(text))

In [76]:
reset_df = emoji_df.reset_index(drop=True)

In [77]:
reset_df.iloc[[2]].text.apply(just_emojis)

2    [😂😂😂]
Name: text, dtype: object

In [78]:
len(reset_df)

174231

In [79]:
# emoji_array = [item for index, item in emoji_df[:2].iterrows()]
emoji_array = [reset_df.loc[[index]].text.apply(just_emojis) for index in range(len(reset_df))]

In [80]:
full_list = []
for item in emoji_array:
    for emoji in item:
        for sinlge in emoji:
            full_list.append(sinlge)

In [81]:
len(full_list)

196678

In [82]:
full_dict = defaultdict(int)

for item in full_list:
    full_dict[item] += 1

number of unique emojis

In [83]:
len(unique(full_dict.keys()))

30495

In [84]:
for item in sorted(full_dict.items(), key=lambda x:x[1], reverse=True)[:100]:
    print(item[0], (float(item[1])/len(full_list))*100)
    


😂 3.8901148069433287
❤ 3.739615005236986
😍 2.627136741272537
😒 2.116149238857422
😏 1.8192171976530165
☺ 1.7922695980231649
👌 1.7480348590081252
😔 1.6941396597484213
😊 1.6473626943532067
😭 1.6310924455200888
😘 1.5370300694536247
😂😂😂 1.4617801686004537
😂😂 1.3966991732679812
😳 1.3107719216180762
😁 1.3006030160973774
😩 1.1973886250622845
💕 1.0545155024964663
🙌 0.8796103275404468
✌ 0.861306297603189
💁 0.8196137849683238
♫ 0.7474145557713623
😌 0.7382625408027335
😋 0.7382625408027335
😑 0.7138571675530563
🎶 0.7041967073083923
😉 0.6782659982306104
🙏 0.6579281871892129
😂😂😂😂 0.6518268438767935
😎 0.5984400898931248
👍 0.5247155248680585
♥ 0.5242070795920235
👏 0.5089537213109753
😞 0.5079368307589054
😍😍😍 0.43624604683797885
😐 0.43522915628590897
😕 0.41946735272882574
😄 0.4082815566560571
♡ 0.4077731113800222
🙈 0.38590996451051973
😜 0.36150459126084256
😴 0.3609961459848076
😍😍 0.36048770070877273
✋ 0.349301904636004
😡 0.34828501408393414
😫 0.34828501408393414
😭😭😭 0.3462512329797944
😢 0.3421836707715149

## JUST FACES

In [102]:
faces = re.compile(u'['
        u'\U0001F600-\U0001F64F]',
#         u'\U0001F680-\U0001F6FF'
#         u'\u2600-\u26FF\u2700-\u27BF]+', 
        re.UNICODE)

def just_person(text):
    return (faces.findall(text))

In [103]:
just_person(u'@Lennayy7 😪')

['😪']

In [99]:
# person_array = [reset_df.loc[[index]].text.apply(just_person) for index in range(len(reset_df))]

In [111]:
# person_array

In [112]:
face_array = [reset_df.loc[[index]].text.apply(just_person) for index in range(len(reset_df))]

face_list = []
for item in face_array:
    for emoji in item:
        for sinlge in emoji:
            face_list.append(sinlge)

In [118]:
print("number of tweets with emojis {0}".format(len(full_list)))
print("number of tweets with faces  {0}".format(len(face_array)))
print("percentage of tweets with emojis with faces {0}%".format(round((float(len(face_array))/len(full_list)*100),1)))

number of tweets with emojis 196678
number of tweets with faces  174231
percentage of tweets with emojis with faces 88.6%


In [113]:
face_dict = defaultdict(int)

for item in face_list:
    face_dict[item] += 1

In [114]:
for item in sorted(face_dict.items(), key=lambda x:x[1], reverse=True)[:100]:
    print(item[0], (float(item[1])/len(full_list))*100)

😂 25.4278566997834
😍 10.767345610591931
😭 8.897792330611457
😘 4.992424165387079
😏 3.678093126836759
😊 3.5616591586247575
😒 3.5255595440262764
😩 3.3867539836687373
😁 3.268794679628632
😔 2.8111939311971854
🙌 2.7323849134117695
😳 2.5635810817681692
🙏 1.7531193117684745
😋 1.6316008907961237
😑 1.397716063820051
😌 1.3814458149869329
😎 1.2797567597799449
😉 1.2558598318063026
😡 1.2467078168376737
😴 1.1577298935315592
🙈 1.1506116596670701
😱 1.0967164604073665
😫 1.057566174152676
😄 0.9670629150184566
😷 0.9075748177223685
🙊 0.8846947803007963
😞 0.8323249168691973
😜 0.8104617699996949
😢 0.759108797120166
😐 0.6782659982306104
😕 0.6411494930800599
😤 0.6162356745543478
😈 0.596406308788985
😪 0.5892880749244959
😝 0.5750516071955175
😃 0.5694587091591332
😖 0.528274641800303
😣 0.5267493059721983
🙋 0.5175972910035692
😅 0.5018354874464862
😬 0.4570923031554114
😛 0.43573760156194385
😓 0.4255686960412451
😠 0.4204842432808957
🙅 0.41285756414037156
😆 0.4011633227915679
😻 0.3894690814427643
😰 0.32540497666236184


In [108]:
len(face_dict)

0

## Exploring Emoji vs no Emoji

In [119]:
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id
0,377652254096228352,37.446100,-121.883557,@Tanner_Cortez hey checkout the website: http:...,Wed Sep 11 04:38:08 +0000 2013,224874450
1,377652255346159616,34.087406,-117.462604,@Lennayy7 😪,Wed Sep 11 04:38:08 +0000 2013,312179473
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,Wed Sep 11 04:38:10 +0000 2013,54351774
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,Wed Sep 11 04:38:11 +0000 2013,343219606
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by @deschutesbee...,Wed Sep 11 04:38:12 +0000 2013,1569395935


In [128]:
def is_emoji(text):
    if highpoints.search(text):
        return 1
    else:
        return 0

In [133]:
def is_face(text):
    if faces.search(text):
        return 1
    else:
        return 0

In [142]:
print(is_emoji(u'@Lennayy7 😪'))
print(is_emoji(u'hello bitch 🔫'))
print(is_emoji(u'so boring'))

1
1
0


In [141]:
print(is_face(u'@Lennayy7 😪'))
print(is_face(u'hello bitch 🔫'))
print(is_face(u'so boring'))

1
0
0


In [143]:
tweet_df["is_emoji"] = tweet_df.text.apply(is_emoji)
tweet_df["is_face"] = tweet_df.text.apply(is_face)

In [146]:
tweet_df[30:40]

,id,lat,lng,text,timeStamp,user_id,is_emoji,is_face
30,377653831934705664,34.125987,-117.990605,😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth,Wed Sep 11 04:44:24 +0000 2013,108769611,1,1
31,377653836804288512,36.920730,-121.744991,@hello_emmelie you know girl. Emmelie let me s...,Wed Sep 11 04:44:25 +0000 2013,1049702996,0,0
32,377653839312465920,37.723981,-122.484290,In the 90's actresses didn't wear bras... #cha...,Wed Sep 11 04:44:26 +0000 2013,38926916,0,0
33,377653845650051072,37.783569,-122.212397,""" you deserve your name on a crown , on a thro...",Wed Sep 11 04:44:27 +0000 2013,230608737,1,0
34,377653846467952640,37.717741,-122.153800,Thank goodness for CPT days 👍,Wed Sep 11 04:44:28 +0000 2013,876663008,1,0
35,377653847717871616,34.185616,-116.399403,Yay my cat is cuddling with me tonight❤,Wed Sep 11 04:44:28 +0000 2013,526041822,1,0
36,377653854109978624,37.348083,-120.569948,Fake people suck...😖,Wed Sep 11 04:44:29 +0000 2013,28348907,1,1
37,377653857146634240,37.620391,-122.090301,Lol now that's it's winter almost... hahaa...,Wed Sep 11 04:44:30 +0000 2013,814226478,0,0
38,377653857394098176,37.577402,-122.323459,I tried to change the subject so I said some c...,Wed Sep 11 04:44:30 +0000 2013,1329849811,0,0
39,377653857364758530,37.758516,-121.980800,@teamroshe just made my day better thanks guys...,Wed Sep 11 04:44:30 +0000 2013,232122031,0,0
